In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [20]:
api_key = ''

In [28]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo', api_key = api_key, temperature = 0.9)
prompt = ChatPromptTemplate.from_template('Can me a creative name for a startup whose product is {product}')
chain = LLMChain(llm = llm, prompt= prompt)

In [32]:
chain.invoke('AI chat bot')

{'product': 'AI chat bot', 'text': 'MindSpeak Technologies'}

In [29]:
prompt1 = ChatPromptTemplate.from_template('Give me a short summary of a company with name {company_name}')
chain1 = LLMChain(llm = llm, prompt=prompt1)

In [33]:
from langchain.chains import SimpleSequentialChain
simpleSeqChain = SimpleSequentialChain(chains = [chain, chain1],verbose = True)

In [34]:
simpleSeqChain.run('Online Dating Website')



> Entering new SimpleSequentialChain chain...
"Heartstrings Connect"
Heartstrings Connect is a social networking platform that focuses on connecting people through shared interests, passions, and causes. Users can create profiles, join groups, and participate in discussions and events to foster meaningful connections and relationships. The company aims to create a supportive and inclusive online community where individuals can express themselves authentically and build genuine connections with others.

> Finished chain.


'Heartstrings Connect is a social networking platform that focuses on connecting people through shared interests, passions, and causes. Users can create profiles, join groups, and participate in discussions and events to foster meaningful connections and relationships. The company aims to create a supportive and inclusive online community where individuals can express themselves authentically and build genuine connections with others.'

In [36]:
from langchain.chains import SequentialChain
seq_prompt1 = ChatPromptTemplate.from_template('Give me the innovative idea for a startup name, whose product is {product}')
seq_chain1 = LLMChain(llm = llm, prompt = seq_prompt1, output_key = 'company_name')
seq_prompt2 = ChatPromptTemplate.from_template('Give me a short summary of the work of a company whose name is {company_name} and product is {product}.')
seq_chain2 = LLMChain(llm = llm, prompt = seq_prompt2, output_key = 'summary')
seq_prompt3 = ChatPromptTemplate.from_template('Give me a tagline for a company whose name is {company_name} and product is {product}.')
seq_chain3 = LLMChain(llm = llm, prompt = seq_prompt3, output_key = 'tagline')
seq_chain = SequentialChain(chains = [seq_chain1, seq_chain2, seq_chain3], input_variables = ['product'], output_variables = ['company_name','summary','tagline'])


In [38]:
seq_chain.invoke('Education Website')

{'product': 'Education Website',
 'company_name': '"LearnLabs"',
 'summary': 'LearnLabs is an innovative company that specializes in providing high-quality educational content through their online education website. Their platform offers a wide range of courses and resources for students of all ages, making learning accessible and engaging. With a team of experienced educators and technology experts, LearnLabs is dedicated to helping individuals reach their full potential through the power of online learning.',
 'tagline': '"Unlocking knowledge with LearnLabs: Your online gateway to learning success."'}

In [39]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [40]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [41]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [42]:
llm = ChatOpenAI(model = 'gpt-3.5-turbo', temperature = 0.0, api_key = api_key)

In [44]:
destination_chains = {}
for p_info in prompt_infos:
    prompt = ChatPromptTemplate.from_template(p_info['prompt_template'])
    llm_chain = LLMChain(llm = llm, prompt = prompt)
    destination_chains[p_info['name']] = llm_chain
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [45]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [46]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [47]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [48]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [49]:
chain.invoke('What is 1+1?')



> Entering new MultiPromptChain chain...
math: {'input': 'What is 1+1?'}
> Finished chain.


{'input': 'What is 1+1?', 'text': '1+1 equals 2.'}

In [50]:
chain.invoke('What is Newtons first law of motion?')



> Entering new MultiPromptChain chain...
physics: {'input': "What is Newton's first law of motion?"}
> Finished chain.


{'input': "What is Newton's first law of motion?",
 'text': "Newton's first law of motion, also known as the law of inertia, states that an object at rest will stay at rest, and an object in motion will stay in motion with a constant velocity unless acted upon by an external force. In simpler terms, an object will not change its state of motion unless a force is applied to it."}

In [51]:
chain.invoke('What is the next movie of Brad Pitt?')



> Entering new MultiPromptChain chain...
None: {'input': 'What is the next movie of Brad Pitt?'}
> Finished chain.


{'input': 'What is the next movie of Brad Pitt?',
 'text': 'The next movie of Brad Pitt is "Bullet Train," set to be released in 2022.'}